# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

c:\Miniconda\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [6]:
import os
from glob import glob

# Write your code below.
price_data= os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(price_data, "**/*.parquet"), recursive= True)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [8]:
# Write your code below.

ddf = dd.read_parquet(parquet_files).set_index("Ticker")
ddf["Close_lag_1"] = ddf.groupby("Ticker")["Close"].shift(1)
ddf["Adj_Close_lag_1"] = ddf.groupby("Ticker")["Adj Close"].shift(1)
ddf["returns"] = (ddf["Close"] / ddf["Close_lag_1"]) - 1
ddf["hi_lo_range"] = ddf["High"] - ddf["Low"]

C:\Users\72xs\AppData\Local\Temp\ipykernel_12448\1433420267.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf["Close_lag_1"] = ddf.groupby("Ticker")["Close"].shift(1)
C:\Users\72xs\AppData\Local\Temp\ipykernel_12448\1433420267.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf["Adj_Close_lag_1"] = ddf.groupby("Ticker")["Adj Close"].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [16]:
# Write your code below.
import pandas as pd
df_pandas =ddf.compute()
df_pandas['rolling_average_return'] = df_pandas.groupby("Ticker")["returns"].transform(lambda x: x.rolling(window=10, min_periods=1).mean())


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

It is not necessary to convert to pandas to calculate the moving average returm. As Desk can handle large datasets, and it can avoid multi-index isue. And also it can perform parallel procssing, Dask distributes computations across multiple cores or even clusters.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.